<a href="https://colab.research.google.com/github/kbsha/neo4j_with_colab/blob/main/graph_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
#!rm /tem/neo4j_with_colab
#!git clone https://github.com/kbsha/neo4j_with_colab
#!git add https://github.com/kbsha/neo4j_with_colab/graph-v1.ipynb

rm: cannot remove '/tem/neo4j_with_colab': Is a directory
fatal: destination path '/tem/neo4j_with_colab' already exists and is not an empty directory.


## Introduction

Created: 17 Feb 2024.   
Author: kibremoges Fenta.   
kibrinfinitytech.netlify.app   


### For Colab users:

If you are using colab, add your credentials here and uncomment these  
and skip to step-1 below.

#### Read the neo4j URI, username, passwd


In [48]:
uri = "neo4j+s://ac7b5a02.databases.neo4j.io"
user = "neo4j"
password = "qDxk-VSgSih4GN_t3dC_S_34R5aidujDa49raagIdqI"

## step-1: install neo4j

In [49]:
%pip install neo4j

## step-2: imports and init

In [50]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable



```
# This is formatted as code
```

## step-3: define helper class

In [51]:
# Import necessary libraries
from neo4j import GraphDatabase

# Define the App class
class App:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def create_node_and_relationship(self, actor_name, usecase_id, rel_name, target_actor_name):
        with self.driver.session() as session:
            # Create a node with use case label
            session.write_transaction(self._create_and_return_node_label, actor_name, usecase_id)

            # Create another node with use case label
            session.write_transaction(self._create_and_return_node_label, target_actor_name, usecase_id)

            # Create a relationship between nodes under the use case
            session.write_transaction(self._create_and_return_rel_label, actor_name, rel_name, target_actor_name, usecase_id)

    def cleanup_db(self):
        with self.driver.session() as session:
            result = session.read_transaction(self._count_nodes_in_db)
            for row in result:
                print("Found {row} actors, deleting! ".format(row=row['count']))
            session.write_transaction(self._cleanup_db)

    @staticmethod
    def _create_and_return_node_label(tx, actor_name, usecase_id):
        query = (
            "CREATE (n1 :" + usecase_id + " { name: $actor_name }) "
            "RETURN n1"
        )
        result = tx.run(query, actor_name=actor_name)
        return result.single()

    @staticmethod
    def _create_and_return_rel_label(tx, actor1_name, rel_name, actor2_name, usecase_id):
        query = (
            "MATCH (n1 :" + usecase_id + " { name: $actor1_name }), "
            "(n2 :" + usecase_id + " { name: $actor2_name }) "
            "CREATE (n1)-[r1:Relation { name: $rel_name }]->(n2) "
            "RETURN n1, r1, n2"
        )
        result = tx.run(query, actor1_name=actor1_name, rel_name=rel_name, actor2_name=actor2_name)
        return result.single()

    @staticmethod
    def _count_nodes_in_db(tx):
        query = (
            "MATCH (n) "
            "RETURN count(n) as count"
        )
        result = tx.run(query)
        try:
            return [{"count": row["count"]}
                    for row in result]
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    @staticmethod
    def _cleanup_db(tx):
        query = (
            "MATCH (n) "
            "DETACH DELETE n"
        )
        tx.run(query)
# Example usage
uri = "neo4j+s://ac7b5a02.databases.neo4j.io"
user = "neo4j"
password = "qDxk-VSgSih4GN_t3dC_S_34R5aidujDa49raagIdqI"
# Create an instance of the App class
app_instance = App(uri, user, password)

# Create a node and a relationship
app_instance.create_node_and_relationship("Actor1", "UseCase1", "Knows", "Actor2")

# Close the driver when done
app_instance.close()

<ipython-input-51-420cd229a064>:15: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_and_return_node_label, actor_name, usecase_id)
<ipython-input-51-420cd229a064>:18: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_and_return_node_label, target_actor_name, usecase_id)
<ipython-input-51-420cd229a064>:21: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_and_return_rel_label, actor_name, rel_name, target_actor_name, usecase_id)


In [52]:
app = App(uri, user, password)

## step-4: instantiate the helper class to DB.
CAUTION: this uses the username/passwd assigned already in step-2 above.

## step-5 clean up DB
#Caution: lose data


In [46]:
app.cleanup_db()

<ipython-input-44-420cd229a064>:25: DeprecationWarning: read_transaction has been renamed to execute_read
  result = session.read_transaction(self._count_nodes_in_db)


Found 4 actors, deleting! 


<ipython-input-44-420cd229a064>:28: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._cleanup_db)
